In [1]:
from sklearn.cluster import KMeans
# common_text에는 파싱된 워드 리스트들이 들어가 있음.
from gensim.test.utils import common_texts
# Doc2Vec이 우리가 텍스트를 사용해서 학습되는 모델(뉴럴넷)이고 
# TaggedDocument가 넘겨주는 텍스트들. 
# 여기서, corpus와 ID들을 함께 넘겨줘야 하는데, 여기서 ID는 tag와 같은 말임
# Q: 여기서, 여러 tag를 함께 넘겨준 다음, 적합한 태그를 찾아주는 방식으로도 처리할 수 있는지 파악하는 것이 필요함. 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# import하기
import json

# Opening JSON file
f = open('./221204_2315.json')
awesome_file = json.load(f)

# TaggedDocument는 해당 corpus의 id를 함께 넘겨주는 것을 말합니다. 
# 아래 코드에서 doc에는 단어의 묶음이, tags에는 해당 문서를 표현하는 태그가 들어가게 됨. 
# 흠, 그렇다면, 이 태그에 고유 id가 아닌 다른 것을 넣어주면 현재 문서에 맞는 tag를 찾아주기도 하나? 
# 이렇게 할 경우, words에는 문서를, tags에는 키워드를 넣고, words에서 tags를 예측하는 짓을 할 수도 있지 않을까? 

common_texts_and_tags = [
    (key, value) for key, value in awesome_file.items()
]

print(len(common_texts_and_tags))


5585


In [2]:

# print("##"*20)
# print("tags and its texts")
# print("##"*20)
# for text, tag in common_texts_and_tags:
#     print(f"tag: {tag}, text: {text}")
#     print("\n")

# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=value, tags=key.split(" ")) for key, value in common_texts_and_tags]

In [3]:
# training. 
# 여러 Parameter들을 사용하여 튜닝할 수 있음. 
model = Doc2Vec(TRAIN_documents, vector_size=100, window=100, epochs=2000, min_count=2, workers=8)
model.build_vocab(TRAIN_documents)

In [4]:
from gensim.test.utils import get_tmpfile
fname = "./my_doc2vec_model"
model.save(fname)

In [5]:
model = Doc2Vec.load(fname)

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize
# python3 -c "import nltk; nltk.download('all')"

#to find the vector of a document which is not in training data
test_data = word_tokenize("text preprocessing representation visualization from zero hero from zero hero installation getting started examples contributions from zero hero texthero python toolkit work with text based dataset quickly effortlessly texthero very simple learn designed used pandas texthero same expressiveness power pandas extensively documented texthero modern conceived programmers decade with little knowledge linguistic think texthero tool help understand work with text based dataset given tabular dataset easy grasp main concept instead given text dataset harder have quick insights into underline data with texthero preprocessing text data mapping into vectors visualizing obtained vector space takes just couple lines texthero include tools preprocess text data offers both solutions also flexible custom solutions natural language processing keyphrases keywords extraction named entity recognition text representation term frequency custom word embeddings vector space analysis clustering means meanshift dbscan hierarchical topic modeling interpretation text visualization vector space visualization place localization maps texthero free open source well documented that what love most hope will find pleasure working with texthero during development hablas espa texthero been developed whole community know hard deal with different tools nltk spacy gensim textblob sklearn that developed texthero simplify things next main milestone provide multilingual support this step need help hablas espa sprechen deutsch fala portugu parli italiano speak another language mentioned here then might help develop multilingual support even haven contributed before just started with contact open github issue there always first time promise will learn knows might help find your developer improving python toolkit provide even better experience your feedback crucial have problem suggestion please open github issue will glad support help beta version texthero community growing fast texthero though still beta version soon faster better version will released will bring some major changes instance give more granular control over pipeline starting from next version preprocessing functions will require argument already tokenized text this will major change once released stable version texthero backward compatibility will respected until this point backward compatibility will present will weaker want part this fast growing movements hesitate contribute contributing installation install texthero bash install texthero under hoods texthero makes multiple machine learning toolkits such gensim nltk spacy scikit learn need install them separately will take care that faster performance make sure have installed spacy version also make sure have recent version python higher best getting started best learn texthero through getting started docs case advanced python user then help texthero should work examples text cleaning representation visualization python import texthero hero import pandas read https github jbesomi texthero master dataset bbcsport text pipe hero clean pipe hero tfidf pipe hero hero scatterplot color topic title sport news text preprocessing means visualization python import texthero hero import pandas read https github jbesomi texthero master dataset bbcsport tfidf text pipe hero clean pipe hero tfidf kmeans labels tfidf pipe hero kmeans clusters astype tfidf pipe hero hero scatterplot color kmeans labels title means sport news simple pipeline text cleaning python import texthero hero import pandas text this ntenc needs cleaned series text this ntenc needs cleane dtype object remove digits python hero remove digits this ntenc needs cleaned dtype object remove digits replaces blocks digits digits string hello will removed want remove digits need blocks false remove types brackets their content python hero remove brackets this ntenc needs cleaned dtype object remove diacritics python hero remove diacritics this sentence needs cleaned dtype object remove punctuation python hero remove punctuation this sentence needs cleaned dtype object remove extra white spaces python hero remove whitespace this sentence needs cleaned dtype object sometimes also want stop words python hero remove stopwords this sentence needs cleaned dtype object texthero composed four modules preprocessing representation visualization preprocessing scope prepare text data further analysis full documentation preprocessing scope provide classic natural language processing tools such named entity noun phrases full documentation representation scope text data into vectors dimensionality reduction supported representation algorithms term frequency count term frequency inverse document frequency tfidf supported clustering algorithms means kmeans density based spatial clustering applications with noise dbscan meanshift meanshift supported dimensionality reduction algorithms principal component analysis distributed stochastic neighbor embedding tsne negative matrix factorization full documentation representation visualization scope summarize main facts regarding text data visualize this module opinionable handy anyone that needs quick solution visualize screen text data instance during text exploratory data analysis supported functions text scatterplot scatterplot most common words words full documentation visualization texthero sometimes just want things done right texthero helps with that helps make things easier give developer more time focus custom requirements believe that cleaning text should just take minute same finding most important part text same representing very pragmatic texthero just goal make developer spare time working with text data pain most cases default pipeline quite good start there always time come back improve previous work contributions texthero been developed member community whole community texthero developers continue exist with precious contribution community your level expertise python does matter anyone help anyone more than welcome contribute expert open issue tell what like dislike texthero what better good creating websites website will soon moved from docusaurus sphinx read open issue there good news website will look like average news need some development adapt this sphinx template needs help good writing probably this most important piece missing texthero more tutorials more getting started guide good writing help start adding page website explain create custom pipeline need help there good python there open issues techie guys which choose have just other questions inquiry drop line jonathanbesomi gmail contributors chronological order selim awwa parth gandhi keefe christian claus bobfang ishan arora vidya cedric conol rich ramalho license license copyright texthero permission hereby granted free charge person obtaining copy this software associated documentation files software deal software without restriction including without limitation rights copy modify merge publish distribute sublicense sell copies software permit persons whom software furnished subject following conditions above copyright notice this permission notice shall included copies substantial portions software software provided without warranty kind express implied including limited warranties merchantability fitness particular purpose noninfringement event shall authors copyright holders liable claim damages other liability whether action contract tort otherwise arising from connection with software other dealings software")
print(model.dv.most_similar(positive=[model.infer_vector(test_data)],topn=5))


[('print', 0.3531869947910309), ('simclr', 0.35159575939178467), ('coding', 0.3453615605831146), ('scikit-image', 0.34471020102500916), ('inspects', 0.3413020372390747)]


In [8]:

# document clustering 

print("##"*30)
print("K-Means Clustering")
print("##"*30)
Clustering_Method = KMeans(n_clusters=2, random_state=0)
X = model.docvecs.vectors_docs # document vector 전체를 가져옴. 
Clustering_Method.fit(X)# fitting 
# 결과를 보면 알겠지만, 생각보다 클러스터링이 잘 되지 않음. 
# 일단은 이것 또한 트레이닝 셋이 적어서 그런 것으로 보임. 
cluster_dict = {i:[] for i in range(0, 2)}
for text_tags, label in zip(common_texts_and_tags, Clustering_Method.labels_):
    text, tags = text_tags
    cluster_dict[label].append(text)
for label, lst in cluster_dict.items():
    print(f"Cluster {label}")
    for x in lst:
        print(x)
    print("--"*30)
print("##"*20)

C:\Users\UC\AppData\Local\Temp\ipykernel_1376\1188204071.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  X = model.docvecs.vectors_docs # document vector 전체를 가져옴.


############################################################
K-Means Clustering
############################################################


AttributeError: 'KeyedVectors' object has no attribute 'vectors_docs'